In [26]:
# importar librerías
import pandas as pd
import nums_from_string
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import nums_from_string

def get_links():
    ser = Service("./chromedriver")
    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")
    links = []
    driver = webdriver.Chrome(options=options, service=ser)
    driver.get("https://www.rightmove.co.uk/property-to-rent/find.html?searchType=RENT&locationIdentifier=REGION%5E87490&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&sortByPriceDescending=&_includeLetAgreed=on&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&letType=&letFurnishType=&houseFlatShare=")
    for i in range(3): #cada uno son 25 links, antes era 8
        apts_links = driver.find_elements(By.CLASS_NAME, "propertyCard-details")
        for a in apts_links:
            links.append(a.find_element(By.CLASS_NAME, "propertyCard-link").get_attribute('href'))
        nb = driver.find_element(By.XPATH, "//button[contains(@class, 'pagination-button pagination-direction pagination-direction--next')]")
        nb.click()
        time.sleep(2)
    df = pd.DataFrame(links)
    df = df.rename(columns={0: 'urls'})
    df.to_csv('urls_rightmove.csv')
    driver.quit()
    print('DONE')
    return

def get_precios_London():
    inicio = time.time()
    prop_dict_list = []
    
    df = pd.read_csv('urls_rightmove.csv')
    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")
    driver = webdriver.Chrome('./chromedriver_mac64/chromedriver', options=options)
    
    for i in df['urls']:
#         print(i)
        prop_dict = {}
        page = driver.get(i)
        precio_class = driver.find_element(By.CLASS_NAME, '_1gfnqJ3Vtd1z40MlC0MzXu')
        precio_span = precio_class.find_element(By.XPATH, 'span')        
        precio_str = precio_span.text
        precio_str = precio_str.replace('.', '').replace(',', '')
        precio_pounds = precio_str[1:-3]
        hay_precio = 1
        hay_tamanho = 1
        try:
            precio_euros = float(precio_pounds)*1.14  # en una de las pestañas el precio era solo letras sin nros(publicidad)
                                                      # por eso evalúa si hay error en float()
        except:
            hay_precio = 0
        try:
            tag = driver.find_element(By.XPATH,"//*[contains(@class,'_3vyydJK3KMwn7-s2BEXJAf')]")
        except:
            hay_tamanho = 0
        else:
            palabra = tag.text[1:-7]
            palabra = palabra.replace(',', '')
            if (len(palabra)>=5):
                sep = palabra.split("-")
                palabra = sep[1]
            sqmeters = float(palabra)
            precio_x_metro = precio_euros/sqmeters
            prop_dict['London'] = precio_x_metro
            if (hay_precio and hay_tamanho):
                prop_dict_list.append(prop_dict)
        time.sleep(1)
    df = pd.DataFrame(prop_dict_list)
    df.to_csv('Precios_London.csv')
    driver.quit()
    fin = time.time()
    print('DONE')
    print('Tiempo:',(fin-inicio)//60,'minutos y',round((fin-inicio)%60) ,'segundos')
    return df

In [19]:
get_links()

DONE


In [25]:
get_precios_London()